In [ ]:
#!pip install torch

In [ ]:
#!pip install torch-geometric
#!pip install -q torch-sparse
#!pip install torch-scatter
#!pip install wandb

In [ ]:
#!pip install pytorch-ignite


# Login to Weights and biases

In [ ]:
!wandb login a42d563f399d54cd749774995c194c7dc83395eb

# Libraries

In [ ]:
from queries import*
from utilities import*
from scipy.spatial import Delaunay
from numpy import linalg
from sklearn import preprocessing
import copy
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import GATConv
import torch.nn as nn
from torch_geometric.nn import global_mean_pool, global_max_pool
import torch
from torch_geometric.data import Dataset
from torch_geometric.data import DataLoader
from torch_geometric.data import Data
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.nn import GATConv

import wandb
from pathlib import Path


In [ ]:
Directory = Path(Path.cwd()).parent
Directory_path = os.path.join(Directory, 'plots', 'GNN')

In [ ]:
plt.rcParams['font.size'] = '12'

In [ ]:
wandb.init(project="GNN-DEEP-MELO")


# Function

In [ ]:
def labeling(string):
    Patients_CR = ['1C', '1Y', '1N', '1J', '2Y', '1O', '1P','3H','1T','3M','3D','2A','2F','1R', '4A', '4C', '4I', '4Q'] 
    if str(string)[0:2] in Patients_CR:
        return 1
    else:
        return 0

In [ ]:
def edges_to_tensor(edge_df):
    
    patch_number_1 = edge_df.patch_number_1.values
    patch_number_2 = edge_df.patch_number_2.values
    
    edges_index = torch.tensor([patch_number_1,patch_number_2])
    
    return edges_index


In [ ]:
def shuffle_and_pick(liste, number):
    
    
    random.shuffle(liste)
    train_indicies = liste[:number]
    test_indicies =liste[number:]
    random.seed()
    print('Train_indices:', train_indicies)
    print('Test_indices:', test_indicies)
    
    return train_indicies, test_indicies

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
            print(input.shape())
            target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

## Data

# Getting labels 

In [ ]:
from pathlib import Path

base_directory = Path(Path.cwd()).parent
Directory_path = os.path.join(base_directory,'data','output')
files = os.listdir(Directory_path)

labels = []
patients_name = []
labels_df = pd.DataFrame({},columns =['patient','label'])

for file in files[:]:
    
    array = file.split('_')
    if len(array)==2:
        condition = array[0]
        PATIENT_ID = array[1]
    else:
        condition = ''
        
    
    if condition == 'DEEPMEL':
        
        labels.append(labeling(PATIENT_ID))
        patients_name.append(PATIENT_ID)
        
labels_df['patient'] = patients_name
labels_df['label'] = labels

labels_df = labels_df.drop_duplicates(['patient']).reset_index().drop(['index'],axis=1)

In [ ]:
len(labels_df)

In [ ]:
labels_df

In [ ]:
label_csv_file = labels_df.to_csv(os.path.join(Directory_path, 'patients_labels.csv'), index= False)

## Defining Graph Dataset

In [ ]:
def get_graph_objects(label_csv_file, patch_features_root_dir, transform_edges = edges_to_tensor):
    
    patient_labels_df = pd.read_csv(label_csv_file)
    
    patient_names = patient_labels_df.patient.values
    
    data_list = []
    
    for index,patient_name in enumerate(patient_names):
        
        base_directory = Path(Path.cwd()).parent
        patient_folder = os.path.join(base_directory,'data','output', 'DEEPMEL_' + patient_name)
        
        patch_features_df = pd.read_csv(os.path.join(patient_folder, 'patch_data_1.csv'))
        patch_features_df = patch_features_df.sort_values(['patch_number']).set_index(['patch_number'])
        patch_features_df = patch_features_df.fillna(0)

        patch_edges_df = pd.read_csv(os.path.join(patient_folder, 'patch_edge_data.csv'))
        patch_edges_df.dropna(how='all', axis=1, inplace=True)
        #patch_features_df.drop(['patch', 'cell_x_position', 'cell_y_position'])
        #print(patch_features_df)

        x = torch.from_numpy(patch_features_df.drop(['patch', 'cell_x_position', 'cell_y_position'], axis=1).values)
        #x = torch.reshape(x, (1, x.size()))
        #print(x.size())
        
        edge_index = transform_edges(patch_edges_df)
        
        y = torch.tensor(patient_labels_df[patient_labels_df.patient == patient_name ].label.values)
        #print(y.size())
        
        data = Data(y=y, x = x, edge_index= edge_index)
        
        data_list.append(data)
        #print(data)
    
    #print(data_list)

    return data_list

In [ ]:
patients_dataset = get_graph_objects(Directory_path + '\\patients_labels.csv',Directory_path )

## Training testing partition

In [ ]:
index_class_1 = list(labels_df[labels_df.label == 1].index)
index_class_0 = list(labels_df[labels_df.label == 0].index)

train_indicies_class_1 , test_indicies_class_1 =  shuffle_and_pick(index_class_1, 14)
train_indicies_class_0 , test_indicies_class_0 =  shuffle_and_pick(index_class_0, 47)

train_indicies = train_indicies_class_1 + train_indicies_class_0
test_indicies = test_indicies_class_1 + test_indicies_class_0 

train_dataset = [patients_dataset[i] for i in train_indicies]
test_dataset = [patients_dataset[i] for i in test_indicies]

In [ ]:
type(patients_dataset[5])

In [ ]:
len(test_dataset)

In [ ]:
BATCH_SIZE = 11

In [ ]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
for batch in train_loader:
    batch

In [ ]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

In [ ]:
for step, data in enumerate(test_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

# Models

# GCN Two layers graph convolutional neural networks 

The graph convolutional operator from the "Semi-supervised Classification with Graph Convolutional Networks" paper
$$
\mathbf{X}^{\prime}=\hat{\mathbf{D}}^{-1 / 2} \hat{\mathbf{A}} \hat{\mathbf{D}}^{-1 / 2} \mathbf{X} \boldsymbol{\Theta}
$$
where $\hat{\mathbf{A}}=\mathbf{A}+\mathbf{I}$ denotes the adjacency matrix with inserted self-loops and $\hat{D}_{i i}=\sum_{j=0} \hat{A}_{i j}$ its diagonal degree matrix. The adjacency matrix can include other values than 1 representing
edge weights via the optional edge_weight tensor.
Its node-wise formulation is given by:
$$
\mathbf{x}_{i}^{\prime}=\boldsymbol{\Theta} \sum_{j \in \mathcal{N}(v) \cup\{i\}} \frac{e_{j, i}}{\sqrt{\hat{d}_{j} \hat{d}_{i}}} \mathbf{x}_{j}
$$
with $\hat{d}_{i}=1+\sum_{j \in \mathcal{N}(i)} e_{j, i}$, where $e_{j, i}$ denotes the edge weight from source node $\mathrm{j}$ to target node $i$ (default: $1.0$ )
PARAMETERS:
- in_channels (int) - Size of each input sample.
- out_channels (int) - Size of each output sample.
- improved (bool, optional) - If set to True, the layer computes $\hat{\mathbf{A}}$ as $\mathbf{A}+2 \mathbf{I}$. (default: False)
- cached (bool, optional) - If set to True, the layer will cache the computation of $\hat{\mathbf{D}}^{-1 / 2} \hat{\mathbf{A}} \hat{\mathbf{D}}^{-1 / 2}$ on first execution, and will use the cached version for further executions. This parameter should only be
set to True in transductive learning scenarios. (default: False)
- add_self_loops (bool, optional) - If set to Fatse, will not add self-loops to the input graph. (default: True)

- normalize (bool, optional) – Whether to add self-loops and compute symmetric normalization coefficients on the fly. (default: True)

- bias (bool, optional) – If set to False, the layer will not learn an additive bias. (default: True)

**kwargs (optional) – Additional arguments of torch_geometric.nn.conv.MessagePassing.

In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self,in_channels, hidden_channels, n_classes, P):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(in_channels, hidden_channels, add_self_loops = True)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, add_self_loops = True)
        self.lin = Linear(hidden_channels, n_classes)

    def forward(self, data, P=0.8):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=P, training=self.training)
        x = self.lin(x)
        
        return x



# Training and evaluation

In [ ]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

In [ ]:
def train():
    model.train()
    LOSS = []
    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data)
        #print(data.y)
        #print(out)# Perform a single forward pass.
        loss = criterion(out, data.y)# Compute the loss.
        LOSS.append(np.double(loss.detach().numpy()))
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()
        
    return LOSS


def valid():

    model.eval()
    TEST_LOSS = []
    
    for data in test_loader:  # Iterate in batches over the training dataset.
        out = model(data)  # Perform a single forward pass.
        loss = criterion(out, data.y)# Compute the loss.
        TEST_LOSS.append(np.double(loss.detach().numpy()))
       
        
    return TEST_LOSS
    
        
        
        

def test(loader):
    model.eval()
   
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    correct = 0
    
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data)  
        pred = out.argmax(dim=1) # Use the class with highest probability.
        correct += int((pred == data.y).sum())
        print('pred:',pred)
        print('true:',data.y)
        print(confusion_matrix(np.array(data.y),np.array(pred)).ravel())
        tn, fp, fn, tp = confusion_matrix(np.array(data.y),np.array(pred), labels=[0,1]).ravel()
        if len(confusion_matrix(np.array(data.y),np.array(pred), labels=[0,1]).ravel()) >1 :
            TP += tp
            FP += fp
            FN += fn
            TN += tn
        else : 
            if (np.array(data.y)[0]==0):
                TN += len(confusion_matrix(np.array(data.y),np.array(pred), labels=[0,1]).ravel()) 
                
            else :
                TP += len(confusion_matrix(np.array(data.y),np.array(pred), labels=[0,1]).ravel())
            
       
        
    #print(TP, FP, FN)
    precision_1 = TP /(TP+FP)
    recall_1 = TP / (TP+FN)
    precision_0 = TN /(TN+FN)
    recall_0 = TN / (TN+FP)
    return correct / len(loader.dataset), precision_1, recall_1,precision_0, recall_0# Derive ratio of correct predictions.

###  GCN : Fixing Model HyperParameters

In [ ]:
BATCH_SIZE = 10


index_class_1 = list(labels_df[labels_df.label == 1].index)
index_class_0 = list(labels_df[labels_df.label == 0].index)

train_indicies_class_1 , test_indicies_class_1 =  shuffle_and_pick(index_class_1, 14)
train_indicies_class_0 , test_indicies_class_0 =  shuffle_and_pick(index_class_0, 47)

train_indicies = train_indicies_class_1 + train_indicies_class_0
test_indicies = test_indicies_class_1 + test_indicies_class_0 

train_dataset = [patients_dataset[i] for i in train_indicies] 
test_dataset = [patients_dataset[i] for i in test_indicies]
#print(train_dataset)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) 
wandb.init(project='DEEP-MLO')

Learning_rate = 0.01
IN_CHANNELS = 64
HIDDEN_CHANNELS = BATCH_SIZE
N_CLASSES = 2
DROPOUT = 0
CLASS_1_WEIGHT = 3
ALPHA = 5
GAMMA = 5 



model = GCN(IN_CHANNELS,HIDDEN_CHANNELS,N_CLASSES, DROPOUT).double()

opt_1 = torch.optim.Adam(model.parameters(), lr=Learning_rate)

opt_2 = torch.optim.SGD(model.parameters(), lr=Learning_rate, momentum = 0.9)

optimizer = opt_1


WEIGHTS = torch.FloatTensor([1,CLASS_1_WEIGHT]).double()
criterion = torch.nn.CrossEntropyLoss(weight = WEIGHTS)
#criterion = FocalLoss(GAMMA,ALPHA)





LOSS_epoch_GCN = []
TOTAL_LOSS = []
TEST_epoch = []
recall_class_0 =[]
precision_class_0 =[]
recall_class_1 =[]
precision_class_1 =[]

Train_accuracy = []
Test_accuracy = []


for epoch in range(1, 201):
    loss = train()
    test_loss = valid()
    print(loss)
    TOTAL_LOSS = TOTAL_LOSS + loss
    LOSS_epoch_GCN.append(np.mean(loss))
    TEST_epoch.append(np.mean(test_loss))
    train_acc = test(train_loader)[0]
    print('test')
    test_acc = test(test_loader)[0]
    recall_1 = test(test_loader)[2]
    precision_1 = test(test_loader)[1]
    precision_0 = test(test_loader)[3]
    recall_0 = test(test_loader)[4]
    
    recall_class_0.append(recall_0)
    recall_class_1.append(recall_1)
    
    precision_class_0.append(precision_0)
    precision_class_1.append(precision_1)
   
    Train_accuracy.append(train_acc)
    Test_accuracy.append(test_acc)



    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, precision_1: {precision_1:.4f}, recall_1: {recall_1:.4f},precision_0: {precision_0:.4f},recall_0: {recall_0:.4f}')
    wandb.log({"loss": np.mean(loss),"Train accuracy": train_acc, "Test acuuracy":test_acc, 'precision_1':precision_1,'recall_1':recall_1,'precision_0': precision_0,'recall_0': recall_0})
    
    
    

In [ ]:
plt.figure(figsize=(15,7))
n = len(TOTAL_LOSS)
plt.plot(range(n),TOTAL_LOSS[:]   )
PLOT_DIRECTORY = os.path.join(base_directory,'plots')
PLOT_PATH = os.path.join(PLOT_DIRECTORY, 'DEEPMEL_' + PATIENT_ID)
plt.xlabel('steps')
plt.ylabel('LOSS')
plt.grid()
plt.title('Training loss' )
file_name = r'Training_loss_model_GCN_(lr='+str(Learning_rate)+'),(DROPOUT='+str(DROPOUT)+'),(CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png'
plt.savefig(os.path.join(PLOT_DIRECTORY ,file_name))

In [ ]:
plt.figure(figsize=(15,7))
n = len(recall_class_0)
x_axis = range(n)
plt.plot(x_axis,recall_class_0, label = 'Recall 0'  )
plt.plot(x_axis,recall_class_1, label = 'Recall 1'  )
plt.plot(x_axis,precision_class_0, label = 'Precision 0'  )
plt.plot(x_axis,precision_class_1, label = 'Precision 1'  )
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.title('Testing metrics')
plt.legend()
plt.grid()
plt.savefig(PLOT_DIRECTORY + 'Test_recall_precision_model GCN (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

In [ ]:
plt.figure(figsize=(15,7))

plt.plot(Train_accuracy[:],label= 'Training Accuracy')
plt.plot(Test_accuracy[:], label= 'Test Accuracy' )



plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()
plt.legend()
plt.title( 'Training- Test accuracy' )
plt.savefig(PLOT_DIRECTORY + 'Train -Test Accuracy _model GCN_ (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png')
#plt.savefig(PLOT_DIRECTORY + 'Train-Test Accuracy: model GCN Best.png')


In [ ]:
plt.figure(figsize=(15,7))
plt.plot(LOSS_epoch_GCN[:],label= 'Training loss'   )
plt.plot(TEST_epoch[:], label= 'validation loss' )


plt.xlabel('steps')
plt.ylabel('LOSS')
plt.grid()
plt.legend()
plt.title( 'Training and validation loss GCN model  ' )
plt.savefig(PLOT_DIRECTORY + 'Training_vs_Validation_loss model GCN (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

## Weights and Biases : Sweeps

In [ ]:

sweep_config = {
  "name" : "GCN_sweep",
  "method" : "grid",
   "metric":{
  "name" : "loss"},
  
  "parameters" : {
    "epochs" : {
      "values" : [200]
    },
    "learning_rate" :{
      "min": 0.0001,
      "max": 0.1
    },
      "batch_size" : {
          "distribution" :'int_uniform',
          "min":1,
          "max":40
      },
      
      "class_1_weight": {
          
          "min":1,
          "max":20
      }
      
      
  }
}

#sweep_id = wandb.sweep(sweep_config,project='DEEP-MLO')

In [ ]:
#def train_1():
#    with wandb.init() as run:
#        config = wandb.config
#        BATCH_SIZE = config["batch_size"]
#        LR = config["learning_rate"]
#        DROPOUT = 0
#        CLASS_1_WEIGHT = config["class_1_weight"]
#        WEIGHTS = torch.FloatTensor([1,CLASS_1_WEIGHT]).double()
        
#        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)    
        
        
#        model = GCN(IN_CHANNELS,HIDDEN_CHANNELS,N_CLASSES, DROPOUT).double()
        
#        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
#        criterion = torch.nn.CrossEntropyLoss(WEIGHTS).double()
    
#        for epoch in range(1, 201):
#            loss = train()
#            avg_loss = np.mean(loss)
#            wandb.log({"loss": np.mean(loss)})
        

#count = 10 # number of runs to execute
#wandb.agent(sweep_id = 'x8o354ox', function=train_1,count=count)

## 10-fold Cross validation  

In [ ]:
N_FOLDS = 10
Learning_rate = 0.01
IN_CHANNELS = 64
HIDDEN_CHANNELS = BATCH_SIZE
N_CLASSES = 2
DROPOUT = 0
CLASS_1_WEIGHT = 3
ALPHA = 5
GAMMA = 5
BATCH_SIZE = 10


index_class_1 = list(labels_df[labels_df.label == 1].index)
index_class_0 = list(labels_df[labels_df.label == 0].index)





WEIGHTS = torch.FloatTensor([1,CLASS_1_WEIGHT]).double()
criterion = torch.nn.CrossEntropyLoss(weight = WEIGHTS)


TRAIN_ACC_BOXPLOT = []
TEST_ACC_BOXPLOT = []
RECALL_1_BOXPLOT = []
RECALL_0_BOXPLOT = []
PRECISION_1_BOXPLOT = []
PRECISION_0_BOXPLOT = []
CONVERGED_TRAINING_LOSS = []
CONVERGED_VALID_LOSS = []


for step in range(N_FOLDS):
    
    print('Fold : ',step)
 
    model = GCN(IN_CHANNELS,HIDDEN_CHANNELS,N_CLASSES, DROPOUT).double()
    criterion = torch.nn.CrossEntropyLoss(weight = WEIGHTS)
    opt_1 = torch.optim.Adam(model.parameters(), lr=Learning_rate)
    opt_2 = torch.optim.SGD(model.parameters(), lr=Learning_rate, momentum = 0.9)
    optimizer = opt_1
    
    train_indicies_class_1 , test_indicies_class_1 =  shuffle_and_pick(index_class_1, 14)
    train_indicies_class_0 , test_indicies_class_0 =  shuffle_and_pick(index_class_0, 47)
    

    train_indicies = train_indicies_class_1 + train_indicies_class_0
    test_indicies = test_indicies_class_1 + test_indicies_class_0 

    train_dataset = [patients_dataset[i] for i in train_indicies] 
    test_dataset = [patients_dataset[i] for i in test_indicies]

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) 
    
  
    
    LOSS_epoch = []
    TOTAL_LOSS = []
    TEST_epoch = []
    recall_class_0 =[]
    precision_class_0 =[]
    recall_class_1 =[]
    precision_class_1 =[]

    Train_accuracy = []
    Test_accuracy = []


    for epoch in range(1, 201):
        loss = train()
        test_loss = valid()
        #print(loss)
        TOTAL_LOSS = TOTAL_LOSS + loss
        LOSS_epoch.append(np.mean(loss))
        TEST_epoch.append(np.mean(test_loss))
        train_acc = test(train_loader)[0]
        #print('test')
        test_acc = test(test_loader)[0]
        recall_1 = test(test_loader)[2]
        precision_1 = test(test_loader)[1]
        precision_0 = test(test_loader)[3]
        recall_0 = test(test_loader)[4]
    
        recall_class_0.append(recall_0)
        recall_class_1.append(recall_1)
    
        precision_class_0.append(precision_0)
        precision_class_1.append(precision_1)
   
        Train_accuracy.append(train_acc)
        Test_accuracy.append(test_acc)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, precision_1: {precision_1:.4f}, recall_1: {recall_1:.4f},precision_0: {precision_0:.4f},recall_0: {recall_0:.4f}')
        
    TRAIN_ACC_BOXPLOT.append(train_acc)
    TEST_ACC_BOXPLOT.append(test_acc)
    RECALL_1_BOXPLOT.append(recall_1)
    RECALL_0_BOXPLOT.append(recall_0)
    PRECISION_1_BOXPLOT.append(precision_1)
    PRECISION_0_BOXPLOT.append(precision_0)
    CONVERGED_TRAINING_LOSS.append(np.mean(loss))
    CONVERGED_VALID_LOSS.append(np.mean(test_loss))  



In [ ]:
losses = pd.DataFrame({},columns =['training_loss', 'validation_loss'])
losses['training_loss'] = CONVERGED_TRAINING_LOSS
losses['validation_loss'] = CONVERGED_VALID_LOSS

In [ ]:
metrics = pd.DataFrame({},columns =['train accuracy', 'test accuracy', 'recall_1', 'precision_1', 'recall_0','precision_0'])
metrics['train accuracy'] = TRAIN_ACC_BOXPLOT
metrics['test accuracy'] = TEST_ACC_BOXPLOT
metrics['recall_1'] = RECALL_1_BOXPLOT
metrics['recall_0'] = RECALL_0_BOXPLOT
metrics['precision_1'] = PRECISION_1_BOXPLOT
metrics['precision_0'] = PRECISION_0_BOXPLOT

base_directory = Path(Path.cwd()).parent
OUTPUT = os.path.join(base_directory,'data','output')

METRICS_PATH = os.path.join(OUTPUT, 'metrics')

metrics.to_csv(METRICS_PATH + '\\GCN.csv', index=False)

In [ ]:
plt.figure(figsize=(15,7))
sns.boxplot(data=metrics)
plt.xlabel('METRICS')
plt.ylabel('VALUES')
plt.grid()
plt.title( 'Boxplots of different metrics for GCN model' )
plt.savefig(PLOT_DIRECTORY + 'BOXPLOTS GCN: (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

# GAT 

The graph attentional operator from the "Graph Attention Networks" paper
$$
\mathbf{x}_{i}^{\prime}=\alpha_{i, i} \boldsymbol{\Theta} \mathbf{x}_{i}+\sum_{j \in \mathcal{N}(i)} \alpha_{i, j} \boldsymbol{\Theta} \mathbf{x}_{j}
$$
where the attention coefficients $\alpha_{i, j}$ are computed as
$$
\alpha_{i, j}=\frac{\exp \left(\operatorname{LeakyReLU}\left(\mathbf{a}^{\top}\left[\boldsymbol{\Theta} \mathbf{x}_{i} \| \boldsymbol{\Theta} \mathbf{x}_{j}\right]\right)\right)}{\sum_{k \in \mathcal{N}(i) \cup\{i\}} \exp \left(\operatorname{LeakyReLU}\left(\mathbf{a}^{\top}\left[\boldsymbol{\Theta} \mathbf{x}_{i} \| \boldsymbol{\Theta} \mathbf{x}_{k}\right]\right)\right)}
$$
PARAMETERS:\
$\quad$ - in_channels (int or tuple) - Size of each input sample. A tuple corresponds to the sizes of source and target dimensionalities.
- out_channels (int) - Size of each output sample.
- heads (int, optional) - Number of multi-head-attentions. (default: 1 )
- concat (bool, optional) - If set to Fatse, the multi-head attentions are
averaged instead of concatenated. (default: True )
- negative_slope (float, optional) - LeakyReLU angle of the negative slope. (default: $0.2$ )
- dropout (float, optional) - Dropout probability of the normalized attention coefficients which exposes each node to a stochastically sampled neighborhood during training. (default: 0 )
- add_self_loops (bool, optional) - If set to False, will not add self-loops
to the input graph. (default: True )
- bias (bool, optional) - If set to False, the layer will not learn an additive
bias. (default: True)
- $^{* *}$ kwargs (optional) - Additional arguments of
torch_geometric.nn. conv.MessagePassing.

In [ ]:
class GAT(nn.Module):
    def __init__(self, hidden_channels, out_channels, first_heads, output_heads, dropout):
        super(GAT, self).__init__()
        self.gc1 = GATConv(64, hidden_channels,
                           heads=first_heads, dropout=dropout)
        
        self.gc2 = GATConv(hidden_channels*first_heads, hidden_channels,
                           heads=output_heads, dropout=dropout)
        
        self.dropout = dropout
        self.lin = Linear(hidden_channels, 2)


    def reset_parameters(self):
        self.gc1.reset_parameters()
        self.gc2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch= data.x, data.edge_index, data.batch
        #x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.gc1(x, edge_index)
        x = F.relu(x)
        #x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.gc2(x, edge_index)
        x = F.relu(x)
        
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        
        return F.log_softmax(x, dim=1)

In [ ]:
BATCH_SIZE = 10


index_class_1 = list(labels_df[labels_df.label == 1].index)
index_class_0 = list(labels_df[labels_df.label == 0].index)

train_indicies_class_1 , test_indicies_class_1 =  shuffle_and_pick(index_class_1, 14)
train_indicies_class_0 , test_indicies_class_0 =  shuffle_and_pick(index_class_0, 47)

train_indicies = train_indicies_class_1 + train_indicies_class_0
test_indicies = test_indicies_class_1 + test_indicies_class_0 

train_dataset = [patients_dataset[i] for i in train_indicies] 
test_dataset = [patients_dataset[i] for i in test_indicies]

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) 
#wandb.init(project='oncology-project')

Learning_rate = 0.01
IN_CHANNELS = 64
HIDDEN_CHANNELS = BATCH_SIZE
N_CLASSES = 2
DROPOUT = 0
CLASS_1_WEIGHT = 3




model = GAT( hidden_channels =BATCH_SIZE, out_channels =2,first_heads=8, output_heads=1, dropout=DROPOUT).double()

opt_1 = torch.optim.Adam(model.parameters(), lr=Learning_rate)

opt_2 = torch.optim.SGD(model.parameters(), lr=Learning_rate, momentum = 0.9)

optimizer = opt_1


WEIGHTS = torch.FloatTensor([1,CLASS_1_WEIGHT]).double()
criterion = torch.nn.CrossEntropyLoss(weight = WEIGHTS)
#criterion = FocalLoss(GAMMA,ALPHA)

LOSS_epoch_GAT = []
TOTAL_LOSS = []
TEST_epoch = []
recall_class_0 =[]
precision_class_0 =[]
recall_class_1 =[]
precision_class_1 =[]

Train_accuracy = []
Test_accuracy = []


for epoch in range(1, 201):
    loss = train()
    test_loss = valid()
    print(loss)
    TOTAL_LOSS = TOTAL_LOSS + loss
    LOSS_epoch_GAT.append(np.mean(loss))
    TEST_epoch.append(np.mean(test_loss))
    train_acc = test(train_loader)[0]
    print('test')
    test_acc = test(test_loader)[0]
    recall_1 = test(test_loader)[2]
    precision_1 = test(test_loader)[1]
    precision_0 = test(test_loader)[3]
    recall_0 = test(test_loader)[4]
    
    recall_class_0.append(recall_0)
    recall_class_1.append(recall_1)
    
    precision_class_0.append(precision_0)
    precision_class_1.append(precision_1)
   
    Train_accuracy.append(train_acc)
    Test_accuracy.append(test_acc)



    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, precision_1: {precision_1:.4f}, recall_1: {recall_1:.4f},precision_0: {precision_0:.4f},recall_0: {recall_0:.4f}')
    #wandb.log({"loss": np.mean(loss),"Train accuracy": train_acc, "Test acuuracy":test_acc, 'precision_1':precision_1,'recall_1':recall_1,'precision_0': precision_0,'recall_0': recall_0})
    
    
    

In [ ]:
plt.figure(figsize=(15,7))
n = len(recall_class_0)
x_axis = range(n)
plt.plot(x_axis,recall_class_0, label = 'Recall 0'  )
plt.plot(x_axis,recall_class_1, label = 'Recall 1'  )
plt.plot(x_axis,precision_class_0, label = 'Precision 0'  )
plt.plot(x_axis,precision_class_1, label = 'Precision 1'  )
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.title('Testing metrics: GAT MODEL')
plt.legend()
plt.grid()
plt.savefig(PLOT_DIRECTORY + 'Test_recall_precision model GAT (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

In [ ]:
plt.figure(figsize=(15,7))
n = len(LOSS_epoch_GAT)
plt.plot(range(n),Train_accuracy[:],label= 'Training Accuracy'   )
plt.plot(range(n),Test_accuracy[:], label= 'Test Accuracy' )
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()
plt.legend()
plt.title( 'Training- Test accuracy: GAT Model' )
plt.savefig(PLOT_DIRECTORY + 'Train -Test Accuracy model GAT (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png')
#plt.savefig(PLOT_DIRECTORY + 'Train-Test Accuracy: model GCN Best.png')

In [ ]:
plt.figure(figsize=(15,7))
n = len(LOSS_epoch_GAT)
plt.plot(range(n),LOSS_epoch_GAT[:],label= 'Training loss'   )
plt.plot(range(n),TEST_epoch[:], label= 'validation loss' )



plt.xlabel('steps')
plt.ylabel('LOSS')
plt.grid()
plt.legend()
plt.title( 'Training loss' )
plt.savefig(PLOT_DIRECTORY + 'Training_vs_Validation_loss model GAT (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

## GAT CROSS VALIDATION

In [ ]:
N_FOLDS = 10
Learning_rate = 0.01
IN_CHANNELS = 64
HIDDEN_CHANNELS = BATCH_SIZE
N_CLASSES = 2
DROPOUT = 0
CLASS_1_WEIGHT = 3
BATCH_SIZE = 10


index_class_1 = list(labels_df[labels_df.label == 1].index)
index_class_0 = list(labels_df[labels_df.label == 0].index)




WEIGHTS = torch.FloatTensor([1,CLASS_1_WEIGHT]).double()
criterion = torch.nn.CrossEntropyLoss(weight = WEIGHTS)


TRAIN_ACC_BOXPLOT = []
TEST_ACC_BOXPLOT = []
RECALL_1_BOXPLOT = []
RECALL_0_BOXPLOT = []
PRECISION_1_BOXPLOT = []
PRECISION_0_BOXPLOT = []
CONVERGED_TRAINING_LOSS = []
CONVERGED_VALID_LOSS = []


for step in range(N_FOLDS):
    
    print('Fold : ',step)
 
    model = GAT( hidden_channels =BATCH_SIZE, out_channels =2,first_heads=8, output_heads=1, dropout=DROPOUT).double()
    criterion = torch.nn.CrossEntropyLoss(weight = WEIGHTS)
    opt_1 = torch.optim.Adam(model.parameters(), lr=Learning_rate)
    opt_2 = torch.optim.SGD(model.parameters(), lr=Learning_rate, momentum = 0.9)
    optimizer = opt_1
    
    train_indicies_class_1 , test_indicies_class_1 =  shuffle_and_pick(index_class_1, 14)
    train_indicies_class_0 , test_indicies_class_0 =  shuffle_and_pick(index_class_0, 47)

    train_indicies = train_indicies_class_1 + train_indicies_class_0
    test_indicies = test_indicies_class_1 + test_indicies_class_0 

    train_dataset = [patients_dataset[i] for i in train_indicies] 
    test_dataset = [patients_dataset[i] for i in test_indicies]

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) 
    
  
    
    LOSS_epoch = []
    TOTAL_LOSS = []
    TEST_epoch = []
    recall_class_0 =[]
    precision_class_0 =[]
    recall_class_1 =[]
    precision_class_1 =[]

    Train_accuracy = []
    Test_accuracy = []


    for epoch in range(1, 201):
        loss = train()
        test_loss = valid()
        #print(loss)
        TOTAL_LOSS = TOTAL_LOSS + loss
        LOSS_epoch.append(np.mean(loss))
        TEST_epoch.append(np.mean(test_loss))
        train_acc = test(train_loader)[0]
        #print('test')
        test_acc = test(test_loader)[0]
        recall_1 = test(test_loader)[2]
        precision_1 = test(test_loader)[1]
        precision_0 = test(test_loader)[3]
        recall_0 = test(test_loader)[4]
    
        recall_class_0.append(recall_0)
        recall_class_1.append(recall_1)
    
        precision_class_0.append(precision_0)
        precision_class_1.append(precision_1)
   
        Train_accuracy.append(train_acc)
        Test_accuracy.append(test_acc)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, precision_1: {precision_1:.4f}, recall_1: {recall_1:.4f},precision_0: {precision_0:.4f},recall_0: {recall_0:.4f}')
        
    TRAIN_ACC_BOXPLOT.append(train_acc)
    TEST_ACC_BOXPLOT.append(test_acc)
    RECALL_1_BOXPLOT.append(recall_1)
    RECALL_0_BOXPLOT.append(recall_0)
    PRECISION_1_BOXPLOT.append(precision_1)
    PRECISION_0_BOXPLOT.append(precision_0)
    CONVERGED_TRAINING_LOSS.append(np.mean(loss))
    CONVERGED_VALID_LOSS.append(np.mean(test_loss))  


In [ ]:
losses_GAT = pd.DataFrame({},columns =['training_loss', 'validation_loss'])
losses_GAT['training_loss'] = CONVERGED_TRAINING_LOSS
losses_GAT['validation_loss'] = CONVERGED_VALID_LOSS


metrics_GAT = pd.DataFrame({},columns =['train accuracy', 'test accuracy', 'recall_1', 'precision_1', 'recall_0','precision_0'])
metrics_GAT['train accuracy'] = TRAIN_ACC_BOXPLOT
metrics_GAT['test accuracy'] = TEST_ACC_BOXPLOT
metrics_GAT['recall_1'] = RECALL_1_BOXPLOT
metrics_GAT['recall_0'] = RECALL_0_BOXPLOT
metrics_GAT['precision_1'] = PRECISION_1_BOXPLOT
metrics_GAT['precision_0'] = PRECISION_0_BOXPLOT

metrics_GAT.to_csv(METRICS_PATH + '\\GAT.csv', index=False)

plt.figure(figsize=(15,7))
sns.boxplot(data=metrics_GAT)
plt.xlabel('METRICS')
plt.ylabel('VALUES')
plt.grid()
plt.title( 'Boxplots of different metrics for GAT model' )
plt.savefig(PLOT_DIRECTORY + 'BOXPLOTS METRICS GAT (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

In [ ]:
plt.figure(figsize=(15,7))
sns.boxplot(data=losses_GAT)
plt.xlabel('Losses')
plt.ylabel('VALUES')
plt.grid()
plt.title( 'Boxplots of different metrics for GAT model' )
plt.savefig(PLOT_DIRECTORY + 'BOXPLOTS LOSSES GAT (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

# SGC : Simple graph convolution

The simple graph convolutional operator from the "Simplifying Graph Convolutional Networks"
paper
$$
\mathbf{X}^{\prime}=\left(\hat{\mathbf{D}}^{-1 / 2} \hat{\mathbf{A}} \hat{\mathbf{D}}^{-1 / 2}\right)^{K} \mathbf{X} \boldsymbol{\Theta}
$$
where $\hat{\mathbf{A}}=\mathbf{A}+\mathbf{I}$ denotes the adjacency matrix with inserted self-loops and $\hat{D}_{i i}=\sum_{j=0} \hat{A}_{i j}$ its diagonal degree matrix. The adjacency matrix can include other values than 1 representing edge weights via the optional edge_weight tensor.
PARAMETERS:
- in_channels (int) - Size of each input sample.
- out_channels (int) - Size of each output sample.
- $\mathrm{K}$ (int, optional) - Number of hops $K$. (default: 1 )
- cached (bool, optional) - If set to True, the layer will cache the computation of $\left(\hat{\mathbf{D}}^{-1 / 2} \hat{\mathbf{A}} \hat{\mathbf{D}}^{-1 / 2}\right)^{K} \mathbf{X}$ on first execution, and will use the cached version for further executions. This parameter should only be set to True in transductive learning scenarios. (default: False)
- add_self_loops (bool, optional) - If set to Fatse, will not add self-loops to the input graph. (default: True )
- bias (bool, optional) - If set to False, the layer will not learn an additive bias. (default: True)
- $^{* *}$ kwargs (optional) - Additional arguments of torch_geometric.nn. conv.MessagePassing.

In [ ]:
from torch_geometric.nn import SGConv


class SGC(nn.Module):
    def __init__(self,hidden_channels, K):
        super(SGC, self).__init__()
        self.gc1 = SGConv(64, hidden_channels, K=K,add_self_loops=True)
        self.gc2 = SGConv(hidden_channels, hidden_channels, K=K,add_self_loops=True)

        self.lin = Linear(hidden_channels, 2)

    def reset_parameters(self):
        self.gc1.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.gc1(x, edge_index)
        x= F.relu(x)
        x = self.gc2(x, edge_index)
        x= F.relu(x)
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return F.log_softmax(x, dim=1)
    
    

In [ ]:
BATCH_SIZE = 10


index_class_1 = list(labels_df[labels_df.label == 1].index)
index_class_0 = list(labels_df[labels_df.label == 0].index)

train_indicies_class_1 , test_indicies_class_1 =  shuffle_and_pick(index_class_1, 14)
train_indicies_class_0 , test_indicies_class_0 =  shuffle_and_pick(index_class_0, 47)

train_indicies = train_indicies_class_1 + train_indicies_class_0
test_indicies = test_indicies_class_1 + test_indicies_class_0 

train_dataset = [patients_dataset[i] for i in train_indicies] 
test_dataset = [patients_dataset[i] for i in test_indicies]

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) 
#wandb.init(project='oncology-project')

Learning_rate = 0.01
IN_CHANNELS = 64
HIDDEN_CHANNELS = BATCH_SIZE
N_CLASSES = 2
DROPOUT = 0
CLASS_1_WEIGHT = 3
K = 2




model = SGC(hidden_channels=BATCH_SIZE,K=K).double()

opt_1 = torch.optim.Adam(model.parameters(), lr=Learning_rate)

opt_2 = torch.optim.SGD(model.parameters(), lr=Learning_rate, momentum = 0.9)

optimizer = opt_1


WEIGHTS = torch.FloatTensor([1,CLASS_1_WEIGHT]).double()
criterion = torch.nn.CrossEntropyLoss(weight = WEIGHTS)
#criterion = FocalLoss(GAMMA,ALPHA)

LOSS_epoch = []
TOTAL_LOSS = []
TEST_epoch = []
recall_class_0 =[]
precision_class_0 =[]
recall_class_1 =[]
precision_class_1 =[]

Train_accuracy = []
Test_accuracy = []


for epoch in range(1, 201):
    loss = train()
    test_loss = valid()
    print(loss)
    TOTAL_LOSS = TOTAL_LOSS + loss
    LOSS_epoch.append(np.mean(loss))
    TEST_epoch.append(np.mean(test_loss))
    train_acc = test(train_loader)[0]
    print('test')
    test_acc = test(test_loader)[0]
    recall_1 = test(test_loader)[2]
    precision_1 = test(test_loader)[1]
    precision_0 = test(test_loader)[3]
    recall_0 = test(test_loader)[4]
    
    recall_class_0.append(recall_0)
    recall_class_1.append(recall_1)
    
    precision_class_0.append(precision_0)
    precision_class_1.append(precision_1)
   
    Train_accuracy.append(train_acc)
    Test_accuracy.append(test_acc)



    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, precision_1: {precision_1:.4f}, recall_1: {recall_1:.4f},precision_0: {precision_0:.4f},recall_0: {recall_0:.4f}')
    #wandb.log({"loss": np.mean(loss),"Train accuracy": train_acc, "Test acuuracy":test_acc, 'precision_1':precision_1,'recall_1':recall_1,'precision_0': precision_0,'recall_0': recall_0})
    
    

In [ ]:
plt.figure(figsize=(15,7))
n = len(recall_class_0)
x_axis = range(n)
plt.plot(x_axis,recall_class_0, label = 'Recall 0'  )
plt.plot(x_axis,recall_class_1, label = 'Recall 1'  )
plt.plot(x_axis,precision_class_0, label = 'Precision 0'  )
plt.plot(x_axis,precision_class_1, label = 'Precision 1'  )
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.title('Testing metrics: SGC MODEL')
plt.legend()
plt.grid()
plt.savefig(PLOT_DIRECTORY + 'Test_recall_precision model SGC (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

In [ ]:
plt.figure(figsize=(15,7))
n = len(LOSS_epoch)
plt.plot(range(n),Train_accuracy[:],label= 'Training Accuracy'   )
plt.plot(range(n),Test_accuracy[:], label= 'Test Accuracy' )
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()
plt.legend()
plt.title( 'Training- Test accuracy: SGC Model' )
plt.savefig(PLOT_DIRECTORY + 'Train -Test Accuracy model SGC (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png')
#plt.savefig(PLOT_DIRECTORY + 'Train-Test Accuracy: model GCN Best.png')

In [ ]:
plt.figure(figsize=(15,7))
n = len(LOSS_epoch)
plt.plot(range(n),LOSS_epoch[:],label= 'Training loss'   )
plt.plot(range(n),TEST_epoch[:], label= 'validation loss' )
plt.xlabel('steps')
plt.ylabel('LOSS')
plt.grid()
plt.legend()
plt.title( 'Training_ Validation loss : SGC model' )
plt.savefig(PLOT_DIRECTORY + 'Training_vs_Validation_loss model SGC (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

In [ ]:
N_FOLDS = 9
Learning_rate = 0.01
IN_CHANNELS = 64
HIDDEN_CHANNELS = BATCH_SIZE
N_CLASSES = 2
DROPOUT = 0
CLASS_1_WEIGHT = 3
BATCH_SIZE = 10


index_class_1 = list(labels_df[labels_df.label == 1].index)
index_class_0 = list(labels_df[labels_df.label == 0].index)




WEIGHTS = torch.FloatTensor([1,CLASS_1_WEIGHT]).double()
criterion = torch.nn.CrossEntropyLoss(weight = WEIGHTS)


TRAIN_ACC_BOXPLOT = []
TEST_ACC_BOXPLOT = []
RECALL_1_BOXPLOT = []
RECALL_0_BOXPLOT = []
PRECISION_1_BOXPLOT = []
PRECISION_0_BOXPLOT = []
CONVERGED_TRAINING_LOSS = []
CONVERGED_VALID_LOSS = []


for step in range(N_FOLDS):
    
    print('Fold : ',step)
 
    model = SGC(hidden_channels=BATCH_SIZE,K=K).double()
    criterion = torch.nn.CrossEntropyLoss(weight = WEIGHTS)
    opt_1 = torch.optim.Adam(model.parameters(), lr=Learning_rate)
    opt_2 = torch.optim.SGD(model.parameters(), lr=Learning_rate, momentum = 0.9)
    optimizer = opt_1
    
    train_indicies_class_1 , test_indicies_class_1 =  shuffle_and_pick(index_class_1, 14)
    train_indicies_class_0 , test_indicies_class_0 =  shuffle_and_pick(index_class_0, 47)

    train_indicies = train_indicies_class_1 + train_indicies_class_0
    test_indicies = test_indicies_class_1 + test_indicies_class_0 

    train_dataset = [patients_dataset[i] for i in train_indicies] 
    test_dataset = [patients_dataset[i] for i in test_indicies]

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True) 
    
  
    
    LOSS_epoch = []
    TOTAL_LOSS = []
    TEST_epoch = []
    recall_class_0 =[]
    precision_class_0 =[]
    recall_class_1 =[]
    precision_class_1 =[]

    Train_accuracy = []
    Test_accuracy = []


    for epoch in range(1, 201):
        loss = train()
        test_loss = valid()
        #print(loss)
        TOTAL_LOSS = TOTAL_LOSS + loss
        LOSS_epoch.append(np.mean(loss))
        TEST_epoch.append(np.mean(test_loss))
        train_acc = test(train_loader)[0]
        #print('test')
        test_acc = test(test_loader)[0]
        recall_1 = test(test_loader)[2]
        precision_1 = test(test_loader)[1]
        precision_0 = test(test_loader)[3]
        recall_0 = test(test_loader)[4]
    
        recall_class_0.append(recall_0)
        recall_class_1.append(recall_1)
    
        precision_class_0.append(precision_0)
        precision_class_1.append(precision_1)
   
        Train_accuracy.append(train_acc)
        Test_accuracy.append(test_acc)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, precision_1: {precision_1:.4f}, recall_1: {recall_1:.4f},precision_0: {precision_0:.4f},recall_0: {recall_0:.4f}')
        
    TRAIN_ACC_BOXPLOT.append(train_acc)
    TEST_ACC_BOXPLOT.append(test_acc)
    RECALL_1_BOXPLOT.append(recall_1)
    RECALL_0_BOXPLOT.append(recall_0)
    PRECISION_1_BOXPLOT.append(precision_1)
    PRECISION_0_BOXPLOT.append(precision_0)
    CONVERGED_TRAINING_LOSS.append(np.mean(loss))
    CONVERGED_VALID_LOSS.append(np.mean(test_loss)) 

In [ ]:
losses_SGC = pd.DataFrame({},columns =['training_loss', 'validation_loss'])
losses_SGC['training_loss'] = CONVERGED_TRAINING_LOSS
losses_SGC['validation_loss'] = CONVERGED_VALID_LOSS


metrics_SGC = pd.DataFrame({},columns =['train accuracy', 'test accuracy', 'recall_1', 'precision_1', 'recall_0','precision_0'])
metrics_SGC['train accuracy'] = TRAIN_ACC_BOXPLOT
metrics_SGC['test accuracy'] = TEST_ACC_BOXPLOT
metrics_SGC['recall_1'] = RECALL_1_BOXPLOT
metrics_SGC['recall_0'] = RECALL_0_BOXPLOT
metrics_SGC['precision_1'] = PRECISION_1_BOXPLOT
metrics_SGC['precision_0'] = PRECISION_0_BOXPLOT

plt.figure(figsize=(15,7))
sns.boxplot(data=metrics_SGC)
plt.xlabel('METRICS')
plt.ylabel('VALUES')
plt.grid()
plt.title( 'Boxplots of different metrics for SGC model' )
plt.savefig(PLOT_DIRECTORY + 'BOXPLOTS METRICS SGC: (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

## SAGEConv

$$
\mathbf{x}_{i}^{\prime}=\mathbf{W}_{1} \mathbf{x}_{i}+\mathbf{W}_{2} \cdot \operatorname{mean}_{j \in \mathcal{N}(i)} \mathbf{x}_{j}
$$
PARAMETERS:
- in_channels (int or tuple) - Size of each input sample. A tuple corresponds to the sizes of source and target dimensionalities.
- out_channels (int) - Size of each output sample.
- normalize (bool, optional) - If set to True, output features will be $\ell_{2}$ normalized, i.e., $\frac{\mathbf{x}_{i}^{\prime}}{\left\|\mathbf{x}_{i}^{\prime}\right\|_{2}}$. (default: False)
- root_weight (bool, optional) - If set to False, the layer will not add transformed root node features to the output. (default: True )
- bias (bool, optional) - If set to False, the layer will not learn an additive bias. (default: True)
$=^{* *} \mathrm{kwargs}$ (optional) - Additional arguments of torch_geometric.nn. conv.MessagePassing.

In [ ]:
from torch_geometric.nn import SAGEConv


class SAGE(nn.Module):
    def __init__(self,hidden_channels,normalize = True):
        super(SAGE, self).__init__()
        self.gc1 = SAGEConv(64, hidden_channels,normalize=normalize)
        self.gc2 = SAGEConv(hidden_channels, hidden_channels,normalize=normalize)
        self.lin = Linear(hidden_channels, 2)

    def reset_parameters(self):
        self.gc1.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.gc1(x, edge_index)
        x= F.relu(x)
        x = self.gc2(x, edge_index)
        x= F.relu(x)
            
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        
        return F.log_softmax(x, dim=1)
    
    

In [ ]:
BATCH_SIZE = 10


index_class_1 = list(labels_df[labels_df.label == 1].index)
index_class_0 = list(labels_df[labels_df.label == 0].index)

train_indicies_class_1 , test_indicies_class_1 =  shuffle_and_pick(index_class_1, 14)
train_indicies_class_0 , test_indicies_class_0 =  shuffle_and_pick(index_class_0, 47)

train_indicies = train_indicies_class_1 + train_indicies_class_0
test_indicies = test_indicies_class_1 + test_indicies_class_0 

train_dataset = [patients_dataset[i] for i in train_indicies] 
test_dataset = [patients_dataset[i] for i in test_indicies]

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) 
wandb.init(project='oncology-project')

Learning_rate = 0.01
IN_CHANNELS = 64
HIDDEN_CHANNELS = BATCH_SIZE
N_CLASSES = 2
DROPOUT = 0
CLASS_1_WEIGHT = 2
K = 2




model = SAGE(hidden_channels=BATCH_SIZE).double()

opt_1 = torch.optim.Adam(model.parameters(), lr=Learning_rate)

opt_2 = torch.optim.SGD(model.parameters(), lr=Learning_rate, momentum = 0.9)

optimizer = opt_1


WEIGHTS = torch.FloatTensor([1,CLASS_1_WEIGHT]).double()
criterion = torch.nn.CrossEntropyLoss(weight = WEIGHTS)
#criterion = FocalLoss(GAMMA,ALPHA)

LOSS_epoch_SAGE = []
TOTAL_LOSS = []
TEST_epoch = []
recall_class_0 =[]
precision_class_0 =[]
recall_class_1 =[]
precision_class_1 =[]

Train_accuracy = []
Test_accuracy = []


for epoch in range(1, 201):
    loss = train()
    test_loss = valid()
    print(loss)
    TOTAL_LOSS = TOTAL_LOSS + loss
    LOSS_epoch_SAGE.append(np.mean(loss))
    TEST_epoch.append(np.mean(test_loss))
    train_acc = test(train_loader)[0]
    print('test')
    test_acc = test(test_loader)[0]
    recall_1 = test(test_loader)[2]
    precision_1 = test(test_loader)[1]
    precision_0 = test(test_loader)[3]
    recall_0 = test(test_loader)[4]
    
    recall_class_0.append(recall_0)
    recall_class_1.append(recall_1)
    
    precision_class_0.append(precision_0)
    precision_class_1.append(precision_1)
   
    Train_accuracy.append(train_acc)
    Test_accuracy.append(test_acc)



    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, precision_1: {precision_1:.4f}, recall_1: {recall_1:.4f},precision_0: {precision_0:.4f},recall_0: {recall_0:.4f}')
    wandb.log({"loss": np.mean(loss),"Train accuracy": train_acc, "Test acuuracy":test_acc, 'precision_1':precision_1,'recall_1':recall_1,'precision_0': precision_0,'recall_0': recall_0})
    
    

In [ ]:
plt.figure(figsize=(15,7))
n = len(recall_class_0)
x_axis = range(n)
plt.plot(x_axis,recall_class_0, label = 'Recall 0'  )
plt.plot(x_axis,recall_class_1, label = 'Recall 1'  )
plt.plot(x_axis,precision_class_0, label = 'Precision 0'  )
plt.plot(x_axis,precision_class_1, label = 'Precision 1'  )
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.title('Testing metrics: SAGE MODEL')
plt.legend()
plt.grid()
plt.savefig(PLOT_DIRECTORY + 'Test_recall_precision model SAGE (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

In [ ]:
plt.figure(figsize=(15,7))
n = len(LOSS_epoch_SAGE)
plt.plot(range(n),Train_accuracy[:],label= 'Training Accuracy'   )
plt.plot(range(n),Test_accuracy[:], label= 'Test Accuracy' )
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()
plt.legend()
plt.title( 'Training- Test accuracy: SAGE Model' )
plt.savefig(PLOT_DIRECTORY + 'Train -Test Accuracy model SAGE (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png')
#plt.savefig(PLOT_DIRECTORY + 'Train-Test Accuracy: model GCN Best.png')

In [ ]:
plt.figure(figsize=(15,7))
n = len(LOSS_epoch)
plt.plot(range(n),LOSS_epoch_SAGE[:],label= 'Training loss'   )
plt.plot(range(n),TEST_epoch[:], label= 'validation loss' )
plt.xlabel('steps')
plt.ylabel('LOSS')
plt.grid()
plt.legend()
plt.title( 'Training_ Validation loss : SAGE CONV model' )
plt.savefig(PLOT_DIRECTORY + 'Training_vs_Validation_loss model SAGE (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

In [ ]:
plt.figure(figsize=(15,7))
a=15

n = len(LOSS_epoch)-a
plt.plot(range(n),LOSS_epoch_SAGE[a:],label= 'SAGE loss'   )
plt.plot(range(n),LOSS_epoch_GAT[a:], label= 'GAT loss' )
plt.plot(range(n),LOSS_epoch_GCN[a:], label= 'GCN loss' )

plt.xlabel('steps')
plt.ylabel('LOSS')
plt.grid()
plt.legend()
plt.title( 'Comparing different training losses : GAT vs SAGE vs GCN' )
plt.savefig(PLOT_DIRECTORY + 'Comparaison Training losses GAT vs SAGE Vs GCN (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

In [ ]:
N_FOLDS = 10
Learning_rate = 0.01
IN_CHANNELS = 64
HIDDEN_CHANNELS = BATCH_SIZE
N_CLASSES = 2
DROPOUT = 0
CLASS_1_WEIGHT = 3
BATCH_SIZE = 10


index_class_1 = list(labels_df[labels_df.label == 1].index)
index_class_0 = list(labels_df[labels_df.label == 0].index)




WEIGHTS = torch.FloatTensor([1,CLASS_1_WEIGHT]).double()
criterion = torch.nn.CrossEntropyLoss(weight = WEIGHTS)


TRAIN_ACC_BOXPLOT = []
TEST_ACC_BOXPLOT = []
RECALL_1_BOXPLOT = []
RECALL_0_BOXPLOT = []
PRECISION_1_BOXPLOT = []
PRECISION_0_BOXPLOT = []
CONVERGED_TRAINING_LOSS = []
CONVERGED_VALID_LOSS = []


for step in range(N_FOLDS):
    
    print('Fold : ',step)
 
    model = SAGE(hidden_channels=BATCH_SIZE).double()

    criterion = torch.nn.CrossEntropyLoss(weight = WEIGHTS)
    opt_1 = torch.optim.Adam(model.parameters(), lr=Learning_rate)
    opt_2 = torch.optim.SGD(model.parameters(), lr=Learning_rate, momentum = 0.9)
    optimizer = opt_1
    
    train_indicies_class_1 , test_indicies_class_1 =  shuffle_and_pick(index_class_1, 14)
    train_indicies_class_0 , test_indicies_class_0 =  shuffle_and_pick(index_class_0, 47)

    train_indicies = train_indicies_class_1 + train_indicies_class_0
    test_indicies = test_indicies_class_1 + test_indicies_class_0 

    train_dataset = [patients_dataset[i] for i in train_indicies] 
    test_dataset = [patients_dataset[i] for i in test_indicies]

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True) 
    
  
    
    LOSS_epoch = []
    TOTAL_LOSS = []
    TEST_epoch = []
    recall_class_0 =[]
    precision_class_0 =[]
    recall_class_1 =[]
    precision_class_1 =[]

    Train_accuracy = []
    Test_accuracy = []


    for epoch in range(1, 201):
        loss = train()
        test_loss = valid()
        #print(loss)
        TOTAL_LOSS = TOTAL_LOSS + loss
        LOSS_epoch.append(np.mean(loss))
        TEST_epoch.append(np.mean(test_loss))
        train_acc = test(train_loader)[0]
        #print('test')
        test_acc = test(test_loader)[0]
        recall_1 = test(test_loader)[2]
        precision_1 = test(test_loader)[1]
        precision_0 = test(test_loader)[3]
        recall_0 = test(test_loader)[4]
    
        recall_class_0.append(recall_0)
        recall_class_1.append(recall_1)
    
        precision_class_0.append(precision_0)
        precision_class_1.append(precision_1)
   
        Train_accuracy.append(train_acc)
        Test_accuracy.append(test_acc)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, precision_1: {precision_1:.4f}, recall_1: {recall_1:.4f},precision_0: {precision_0:.4f},recall_0: {recall_0:.4f}')
        
    TRAIN_ACC_BOXPLOT.append(train_acc)
    TEST_ACC_BOXPLOT.append(test_acc)
    RECALL_1_BOXPLOT.append(recall_1)
    RECALL_0_BOXPLOT.append(recall_0)
    PRECISION_1_BOXPLOT.append(precision_1)
    PRECISION_0_BOXPLOT.append(precision_0)
    CONVERGED_TRAINING_LOSS.append(np.mean(loss))
    CONVERGED_VALID_LOSS.append(np.mean(test_loss)) 

In [ ]:
losses_SAGE = pd.DataFrame({},columns =['training_loss', 'validation_loss'])
losses_SAGE['training_loss'] = CONVERGED_TRAINING_LOSS
losses_SAGE['validation_loss'] = CONVERGED_VALID_LOSS


metrics_SAGE = pd.DataFrame({},columns =['train accuracy', 'test accuracy', 'recall_1', 'precision_1', 'recall_0','precision_0'])
metrics_SAGE['train accuracy'] = TRAIN_ACC_BOXPLOT
metrics_SAGE['test accuracy'] = TEST_ACC_BOXPLOT
metrics_SAGE['recall_1'] = RECALL_1_BOXPLOT
metrics_SAGE['recall_0'] = RECALL_0_BOXPLOT
metrics_SAGE['precision_1'] = PRECISION_1_BOXPLOT
metrics_SAGE['precision_0'] = PRECISION_0_BOXPLOT

METRICS_PATH = os.path.join(OUTPUT, 'metrics')

metrics_SAGE.to_csv(METRICS_PATH + '\\SAGE.csv', index=False)


plt.figure(figsize=(15,7))
sns.boxplot(data=metrics_SAGE)
plt.xlabel('METRICS')
plt.ylabel('VALUES')
plt.grid()
plt.title( 'Boxplots of different metrics for SAGE model' )
plt.savefig(PLOT_DIRECTORY + 'BOXPLOTS METRICS SAGE (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

## Comparing loss function on the same split

In [ ]:
BATCH_SIZE = 10


index_class_1 = list(labels_df[labels_df.label == 1].index)
index_class_0 = list(labels_df[labels_df.label == 0].index)

train_indicies_class_1 , test_indicies_class_1 =  shuffle_and_pick(index_class_1, 14)
train_indicies_class_0 , test_indicies_class_0 =  shuffle_and_pick(index_class_0, 47)

train_indicies = train_indicies_class_1 + train_indicies_class_0
test_indicies = test_indicies_class_1 + test_indicies_class_0 

train_dataset = [patients_dataset[i] for i in train_indicies] 
test_dataset = [patients_dataset[i] for i in test_indicies]

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) 
#wandb.init(project='oncology-project')

Learning_rate = 0.01
IN_CHANNELS = 64
HIDDEN_CHANNELS = BATCH_SIZE
N_CLASSES = 2
DROPOUT = 0
CLASS_1_WEIGHT = 3





WEIGHTS = torch.FloatTensor([1,CLASS_1_WEIGHT]).double()
criterion = torch.nn.CrossEntropyLoss(weight = WEIGHTS)

In [ ]:
model = GCN(IN_CHANNELS,HIDDEN_CHANNELS,N_CLASSES, DROPOUT).double()
opt_1 = torch.optim.Adam(model.parameters(), lr=Learning_rate)

opt_2 = torch.optim.SGD(model.parameters(), lr=Learning_rate, momentum = 0.9)

optimizer = opt_1
LOSS_epoch_GCN = []
TOTAL_LOSS = []
TEST_epoch_GCN = []
recall_class_0 =[]
precision_class_0 =[]
recall_class_1 =[]
precision_class_1 =[]

Train_accuracy_GCN = []
Test_accuracy_GCN = []


for epoch in range(1, 201):
    loss = train()
    test_loss = valid()
    print(loss)
    TOTAL_LOSS = TOTAL_LOSS + loss
    LOSS_epoch_GCN.append(np.mean(loss))
    TEST_epoch_GCN.append(np.mean(test_loss))
    train_acc = test(train_loader)[0]
    print('test')
    test_acc = test(test_loader)[0]
    recall_1 = test(test_loader)[2]
    precision_1 = test(test_loader)[1]
    precision_0 = test(test_loader)[3]
    recall_0 = test(test_loader)[4]
    
    recall_class_0.append(recall_0)
    recall_class_1.append(recall_1)
    
    precision_class_0.append(precision_0)
    precision_class_1.append(precision_1)
   
    Train_accuracy_GCN.append(train_acc)
    Test_accuracy_GCN.append(test_acc)



    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, precision_1: {precision_1:.4f}, recall_1: {recall_1:.4f},precision_0: {precision_0:.4f},recall_0: {recall_0:.4f}')
    wandb.log({"loss": np.mean(loss),"Train accuracy": train_acc, "Test acuuracy":test_acc, 'precision_1':precision_1,'recall_1':recall_1,'precision_0': precision_0,'recall_0': recall_0})
    
    
    

In [ ]:
plt.figure(figsize=(15,7))
n = len(LOSS_epoch_GCN)
plt.plot(range(n),Train_accuracy_GCN[:],label= 'Training Accuracy'   )
plt.plot(range(n),Test_accuracy_GCN[:], label= 'Test Accuracy' )
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()
plt.legend()
plt.title( 'Training- Test accuracy: GCN Model' )
plt.savefig(PLOT_DIRECTORY + 'Train -Test Accuracy model GCN (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png')
#plt.savefig(PLOT_DIRECTORY + 'Train-Test Accuracy: model GCN Best.png')

In [ ]:
plt.figure(figsize=(15,7))
n = len(LOSS_epoch_GCN)
plt.plot(range(n),LOSS_epoch_GCN[:],label= 'Training loss'   )
plt.plot(range(n),TEST_epoch_GCN[:], label= 'validation loss' )
plt.xlabel('steps')
plt.ylabel('LOSS')
plt.grid()
plt.legend()
plt.title( 'Training_ Validation loss : GCN model' )
plt.savefig(PLOT_DIRECTORY + 'Training_vs_Validation_loss model GCN (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

In [ ]:
model = GAT( hidden_channels =BATCH_SIZE, out_channels =2,first_heads=8, output_heads=1, dropout=DROPOUT).double()

opt_1 = torch.optim.Adam(model.parameters(), lr=Learning_rate)

opt_2 = torch.optim.SGD(model.parameters(), lr=Learning_rate, momentum = 0.9)

optimizer = opt_1


LOSS_epoch_GAT = []
TOTAL_LOSS = []
TEST_epoch_GAT = []
recall_class_0 =[]
precision_class_0 =[]
recall_class_1 =[]
precision_class_1 =[]

Train_accuracy_GAT = []
Test_accuracy_GAT = []


for epoch in range(1, 201):
    loss = train()
    test_loss_GAT = valid()
    print(loss)
    TOTAL_LOSS = TOTAL_LOSS + loss
    LOSS_epoch_GAT.append(np.mean(loss))
    TEST_epoch_GAT.append(np.mean(test_loss))
    train_acc = test(train_loader)[0]
    print('test')
    test_acc = test(test_loader)[0]
    recall_1 = test(test_loader)[2]
    precision_1 = test(test_loader)[1]
    precision_0 = test(test_loader)[3]
    recall_0 = test(test_loader)[4]
    
    recall_class_0.append(recall_0)
    recall_class_1.append(recall_1)
    
    precision_class_0.append(precision_0)
    precision_class_1.append(precision_1)
   
    Train_accuracy_GAT.append(train_acc)
    Test_accuracy_GAT.append(test_acc)



    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, precision_1: {precision_1:.4f}, recall_1: {recall_1:.4f},precision_0: {precision_0:.4f},recall_0: {recall_0:.4f}')
    wandb.log({"loss": np.mean(loss),"Train accuracy": train_acc, "Test acuuracy":test_acc, 'precision_1':precision_1,'recall_1':recall_1,'precision_0': precision_0,'recall_0': recall_0})
    
    
    

In [ ]:
plt.figure(figsize=(15,7))
n = len(LOSS_epoch_GAT)
plt.plot(range(n),Train_accuracy_GAT[:],label= 'Training Accuracy'   )
plt.plot(range(n),Test_accuracy_GAT[:], label= 'Test Accuracy' )
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()
plt.legend()
plt.title( 'Training- Test accuracy: GAT Model' )
plt.savefig(PLOT_DIRECTORY + 'Train -Test Accuracy model GAT (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png')
#plt.savefig(PLOT_DIRECTORY + 'Train-Test Accuracy: model GCN Best.png')

In [ ]:
plt.figure(figsize=(15,7))
n = len(LOSS_epoch_GAT)
plt.plot(range(n),LOSS_epoch_GAT[:],label= 'Training loss'   )
plt.plot(range(n),TEST_epoch_GAT[:], label= 'validation loss' )
plt.xlabel('steps')
plt.ylabel('LOSS')
plt.grid()
plt.legend()
plt.title( 'Training_ Validation loss : GAT model' )
plt.savefig(PLOT_DIRECTORY + 'Training_vs_Validation_loss model GAT (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

In [ ]:
model = SAGE(hidden_channels=BATCH_SIZE).double()

opt_1 = torch.optim.Adam(model.parameters(), lr=Learning_rate)

opt_2 = torch.optim.SGD(model.parameters(), lr=Learning_rate, momentum = 0.9)

optimizer = opt_1

LOSS_epoch_SAGE = []
TOTAL_LOSS = []
TEST_epoch_SAGE = []
recall_class_0 =[]
precision_class_0 =[]
recall_class_1 =[]
precision_class_1 =[]

Train_accuracy_SAGE = []
Test_accuracy_SAGE= []


for epoch in range(1, 201):
    loss = train()
    test_loss = valid()
    print(loss)
    TOTAL_LOSS = TOTAL_LOSS + loss
    LOSS_epoch_SAGE.append(np.mean(loss))
    TEST_epoch_SAGE.append(np.mean(test_loss))
    train_acc = test(train_loader)[0]
    print('test')
    test_acc = test(test_loader)[0]
    recall_1 = test(test_loader)[2]
    precision_1 = test(test_loader)[1]
    precision_0 = test(test_loader)[3]
    recall_0 = test(test_loader)[4]
    
    recall_class_0.append(recall_0)
    recall_class_1.append(recall_1)
    
    precision_class_0.append(precision_0)
    precision_class_1.append(precision_1)
   
    Train_accuracy_SAGE.append(train_acc)
    Test_accuracy_SAGE.append(test_acc)



    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, precision_1: {precision_1:.4f}, recall_1: {recall_1:.4f},precision_0: {precision_0:.4f},recall_0: {recall_0:.4f}')
    wandb.log({"loss": np.mean(loss),"Train accuracy": train_acc, "Test acuuracy":test_acc, 'precision_1':precision_1,'recall_1':recall_1,'precision_0': precision_0,'recall_0': recall_0})
    
    

In [ ]:
plt.figure(figsize=(15,7))
n = len(LOSS_epoch_SAGE)
plt.plot(range(n),Train_accuracy_SAGE[:],label= 'Training Accuracy'   )
plt.plot(range(n),Test_accuracy_SAGE[:], label= 'Test Accuracy' )
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()
plt.legend()
plt.title( 'Training- Test accuracy: SAGE Model' )
plt.savefig(PLOT_DIRECTORY + 'Train -Test Accuracy model SAGE (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png')
#plt.savefig(PLOT_DIRECTORY + 'Train-Test Accuracy: model GCN Best.png')

In [ ]:
plt.figure(figsize=(15,7))
n = len(LOSS_epoch_GCN)
plt.plot(range(n),LOSS_epoch_GCN[:],label= 'Training loss'   )
plt.plot(range(n),TEST_epoch_GCN[:], label= 'validation loss' )
plt.xlabel('steps')
plt.ylabel('LOSS')
plt.grid()
plt.legend()
plt.title( 'Training_ Validation loss : GCN model' )
plt.savefig(PLOT_DIRECTORY + 'Training_vs_Validation_loss model GCN (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png') 

In [ ]:
plt.figure(figsize=(15,7))
a=100
n = len(LOSS_epoch_GAT)
x = np.linspace(a,n,n-a)


plt.plot(x,LOSS_epoch_SAGE[a:],label= 'SAGE loss')
plt.plot(x,LOSS_epoch_GAT[a:], label= 'GAT loss' )
plt.plot(x,LOSS_epoch_GCN[a:], label= 'GCN loss' )

plt.xlabel('steps')
plt.ylabel('LOSS')
plt.grid()
plt.legend()
plt.title( 'Comparing different training losses : GAT vs SAGE vs GCN' )
plt.savefig(PLOT_DIRECTORY + 'Comparaison Training losses GAT vs SAGE Vs GCN: (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+')'+str(a)+'.png') 

In [ ]:
plt.figure(figsize=(15,7))
a=100
n = len(LOSS_epoch_GAT)
x = np.linspace(a,n,n-a)

plt.plot(x,LOSS_epoch_SAGE[a:],label= 'SAGE loss'   )
plt.plot(x,LOSS_epoch_GAT[a:], label= 'GAT loss' )

plt.xlabel('steps')
plt.ylabel('LOSS')
plt.grid()
plt.legend()
plt.title( 'Comparing different training losses : GAT vs SAGE ' )
plt.savefig(PLOT_DIRECTORY + 'Comparaison Training losses GAT vs SAGE: (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+')'+str(a)+'.png') 

In [ ]:
plt.figure(figsize=(15,7))
n = len(LOSS_epoch_GCN)
plt.plot(range(n),Train_accuracy_GCN[:],label= 'Training Accuracy'   )
plt.plot(range(n),Test_accuracy_GCN[:], label= 'Test Accuracy' )
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()
plt.legend()
plt.title( 'Training- Test accuracy: GCN Model' )
plt.savefig(PLOT_DIRECTORY + 'Train-Test Accuracy model GCN  (lr ='+str(Learning_rate)+') , (DROPOUT='+str(DROPOUT)+'), (CLASS_1_WEIGHT='+str(CLASS_1_WEIGHT)+').png')
#plt.savefig(PLOT_DIRECTORY + 'Train-Test Accuracy: model GCN Best.png')